<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/submissions/Arabic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install focal-loss
!pip install keras-tcn==2.8.3

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Import lib

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential,Model
from keras.layers import Embedding,CuDNNGRU,CuDNNLSTM,Dense,Dropout,Bidirectional,BatchNormalization,GlobalMaxPooling1D,Flatten, GlobalAveragePooling1D, MaxPooling1D,SpatialDropout1D,Input,Activation,concatenate,Conv1D
from keras.optimizers import RMSprop,Adam,Adadelta
from keras.initializers import Constant
from sklearn.utils import class_weight
from sklearn.metrics import classification_report,f1_score
import keras
import csv
import collections
import numpy as np
from focal_loss import BinaryFocalLoss
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)


Using TensorFlow backend.


# Load data

In [4]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Arabic/

/content/drive/My Drive/dataset/OffenseEval2020/data/Arabic


In [5]:
ls

cc.ar.300.vec             offenseval-ar-test-tweets-v1.tsv
cc.ar.300.vec.gz          offenseval-ar-training-v1.tsv
offenseval-ar-dev-v1.tsv  readme-data-ar.txt


In [6]:
train=pd.read_csv( 'offenseval-ar-training-v1.tsv',sep='\t',quoting=csv.QUOTE_NONE,encoding='utf-8')
train.drop(train.tail(n=1).index,inplace=True)
train["tweet"]= train["tweet"].str.replace('@USER', "") 
train['tweet']=train['tweet'].str.replace('\d+', '')
train['tweet']=train['tweet'].str.replace('URL', '')
train["tweet"]= train["tweet"].str.lower()
train=train.sample(frac=1)
print(len(train))
train.head()

6999


,id,tweet,subtask_a
1763,1764,صباح ومساء الحب والعشق والموده والرحمة والغرام...,NOT
231,232,rt : طلتّك يا ملح الايام يا الوجه الصبوح <lf>ت...,NOT
5193,5194,rt : يا مُنى الدنيا يا أول وأخر طموح <lf>يالجو...,NOT
4240,4241,يا بشر يا ناس محمد علي تركي وأول حاكم مصري م...,NOT
2065,2066,"rt : يا رب , يا حنان يا منان , إجعل الجزائر في...",NOT


In [7]:
val=pd.read_csv( 'offenseval-ar-dev-v1.tsv',sep='\t',quoting=csv.QUOTE_NONE,encoding='utf-8')
val.drop(val.tail(n=1).index,inplace=True)
val["tweet"]= val["tweet"].str.replace('@USER', "") 
val['tweet']=val['tweet'].str.replace('\d+', '')
val['tweet']=val['tweet'].str.replace('URL', '')
val["tweet"]= val["tweet"].str.lower()
val=val.sample(frac=1)
print(len(val))
val.head()

999


,id,tweet,subtask_a
749,7750,يا زياد يا منصور اول ما شوفتك النهارده في الت...,NOT
5,7006,يا جابر يا بنجرجي يا مرتزق يا حثالة الحمدين يا...,OFF
244,7245,اسأل الله العظيم رب العرش العظيم ان يرزقه الصح...,NOT
479,7480,rt : يا ضحكة قلبّي وأغلى محبيني يا بسمةَ صبّاح...,NOT
772,7773,يا سامح يا شكري اخبار خيارات وإرادة الشعب المص...,NOT


In [9]:
test=pd.read_csv( 'offenseval-ar-test-tweets-v1.tsv',sep='\t',quoting=csv.QUOTE_NONE,encoding='utf-8')
test["tweet"]= test["tweet"].str.replace('@USER', "") 
test['tweet']=test['tweet'].str.replace('\d+', '')
test['tweet']=test['tweet'].str.replace('URL', '')
test['tweet']=test['tweet'].str.lower()
print(test.head())
ids=test['id']
test=test['tweet']
print(len(test))

     id                                              tweet
0  8001   اما انت تقعد طول عمرك لا مبدا ولا راي ثابت يا...
1  8002    بتخاف نسوانك يزعلوا ولا ايه 😂 اه يا هلفوت يا...
2  8003  rt : يا عـسانـى نـبـقى يا عـمري حـبايـب وحـبنـ...
3  8004  rt : باقي البيان وينو ما شفنه يا برهان <lf>وري...
4  8005    اللهم انت الشافي المعافي اشفيه وجميع مرضى ال...
2000


#Label Encoding

In [0]:
y_train=train['subtask_a']
train=train['tweet']

In [0]:
y_val=val['subtask_a']
val=val['tweet']

In [0]:
le=LabelEncoder()
y_val=le.fit_transform(y_val)
y_train=le.fit_transform(y_train)



In [57]:
y_val[0],y_train[0]

(0, 0)

# Common Parameters

In [0]:
batch_size=64
opt=Adam()

## Cylic learning rate

In [0]:
# https://www.kaggle.com/hireme/fun-api-keras-f1-metric-cyclical-learning-rate/code
from keras.callbacks import Callback
class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

In [0]:
clr = CyclicLR(base_lr=0.001, max_lr=0.005,
                        step_size=4., mode='exp_range',
                        gamma=0.99994)

# Tokenize data

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(train)
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index


In [61]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

10001


In [0]:
train_sequences = sequence.pad_sequences(sequences,maxlen=max_len,padding='post')

In [0]:
val_sequences = tokenize.texts_to_sequences(val)
val_sequences = sequence.pad_sequences(val_sequences,maxlen=max_len,padding='post')

In [0]:
test_sequences = tokenize.texts_to_sequences(test)
test_sequences = sequence.pad_sequences(test_sequences,maxlen=max_len,padding='post')

In [76]:
len(test_sequences)

2000

# CharLevel Tokenization

In [0]:
#https://github.com/BrambleXu/nlp-beginner-guide-keras/blob/master/char-level-cnn/char_cnn.py
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences

max_words = 10000 #frequency of words to be kept
max_len = 200
#ي	و	ه	ن	م	ل	ك	ق	ف	غ	ع	ظ	ط	ض	ص	ش	س	ز	ر	ذ	د	خ	ح	ج	ث	ت	ب	ا
ch_tokenize = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
ch_tokenize.fit_on_texts(train)

alphabet = "ي	و	ه	ن	م	ل	ك	ق	ف	غ	ع	ظ	ط	ض	ص	ش	س	ز	ر	ذ	د	خ	ح	ج	ث	ت	ب	ا,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1

# Use char_dict to replace the ch_tokenize.word_index
ch_tokenize.word_index = char_dict.copy()
# Add 'UNK' to the vocabulary
ch_tokenize.word_index[ch_tokenize.oov_token] = max(char_dict.values()) + 1

train_char = ch_tokenize.texts_to_sequences(train)
val_char = ch_tokenize.texts_to_sequences(val)
text_char = ch_tokenize.texts_to_sequences(test)

# Padding
train_data = pad_sequences(train_char, maxlen=max_len, padding='post')
val_data = pad_sequences(val_char, maxlen=max_len, padding='post')
test_data = pad_sequences(text_char, maxlen=max_len, padding='post')

# Embeddings

In [0]:
embed_size=300
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def build_matrix(embedding_path, word_index):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

    nb_words = min(max_words, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [30]:
embeddings=build_matrix('cc.ar.300.vec', word_index)
embeddings.shape

(10001, 300)

# Base Models

## Count Vectorizer

In [31]:
train[0]

'الحمدلله يارب فوز مهم يا زمالك.. كل الدعم ليكم يا رجالة ⚪🔴<lf>الدوري يا زمالك .. الدوري يا زمالك<lf>#صدارة_بس'

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
count_vectorizer.fit(train)
X_train_cv = count_vectorizer.transform(train)
X_val_cv  = count_vectorizer.transform(val)
X_test_cv=  count_vectorizer.transform(test)

In [91]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
def logistic_param_selection(X, y, nfolds):
    C= [0.1, 1,3,5,8, 10,12,15,17,20]
    param_grid = {'C': C}
    grid_search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=nfolds,scoring='f1_macro')
    grid_search.fit(X, y)
    print(grid_search.best_params_)
    return grid_search.best_score_
count_vectorizer.fit(train)
train_cv = count_vectorizer.transform(train)
logistic_param_selection(train_cv,y_train,5)

{'C': 17}


0.7771470590236851

In [0]:
count_vectorizer.transform(train)

In [92]:
from sklearn.linear_model import LogisticRegression
cv_classifier = LogisticRegression(solver='lbfgs',C=17,max_iter=500,class_weight='balance')
cv_classifier.fit(X_train_cv, y_train)
y_pred = cv_classifier.predict(X_val_cv)
print(classification_report(y_val, y_pred))
cv_f1=f1_score(y_val, y_pred,average='macro')

              precision    recall  f1-score   support

           0       0.90      0.98      0.94       820
           1       0.82      0.50      0.62       179

    accuracy                           0.89       999
   macro avg       0.86      0.74      0.78       999
weighted avg       0.89      0.89      0.88       999



## TF IDF word vectorizer

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 4),
    max_features=10000)
word_vectorizer.fit(x_train)
train_word_features = word_vectorizer.transform(train)
val_word_features = word_vectorizer.transform(val)
test_word_features = word_vectorizer.transform(test)

In [93]:
from sklearn.linear_model import LogisticRegression
tfw_classifier = LogisticRegression(solver='lbfgs',max_iter=500,C=17)
tfw_classifier.fit(train_word_features, y_train)
y_pred = tfw_classifier.predict(val_word_features)
print(classification_report(y_val, y_pred))
tfw_f1=f1_score(y_val, y_pred,average='macro')

              precision    recall  f1-score   support

           0       0.91      0.96      0.94       820
           1       0.75      0.59      0.66       179

    accuracy                           0.89       999
   macro avg       0.83      0.77      0.80       999
weighted avg       0.88      0.89      0.89       999



## TF IDF char vectorizer

In [0]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 6),
    max_features=30000)
char_vectorizer.fit(x_train)
train_char_features = char_vectorizer.transform(train)
val_char_features = char_vectorizer.transform(val)
test_char_features = char_vectorizer.transform(test)

In [41]:
tfc_classifier = LogisticRegression(solver='lbfgs',max_iter=500,C=15)
tfc_classifier.fit(train_char_features, y_train)
y_pred = tfc_classifier.predict(val_char_features)
print(classification_report(y_val, y_pred))
tfc_f1=f1_score(y_val, y_pred,average='macro')

              precision    recall  f1-score   support

         NOT       0.92      0.98      0.95       820
         OFF       0.88      0.59      0.71       179

    accuracy                           0.91       999
   macro avg       0.90      0.79      0.83       999
weighted avg       0.91      0.91      0.90       999



# Models

##GRU

In [0]:
def model_gru():
  inp = Input(shape=(max_len,))
  x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  x = SpatialDropout1D(0.1)(x)
  x = Bidirectional(CuDNNLSTM(50, return_sequences=True))(x)
  x, x_h, x_c = Bidirectional(CuDNNGRU(50, return_sequences=True, return_state = True))(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  conc = concatenate([avg_pool, x_h, max_pool])
  outp = Dense(1, activation="sigmoid")(conc)    
  model = Model(inputs=inp, outputs=outp)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer=opt, metrics=['accuracy'])
  return model

## GRU attention

In [0]:
#https://github.com/zake7749/DeepToxic
from keras.layers import Layer,Lambda
from keras import initializers
from keras.engine import InputSpec, Layer
from keras import backend as K

class AttentionWeightedAverage(Layer):
    """
    Computes a weighted average of the different channels across timesteps.
    Uses 1 parameter pr. channel to compute the attention value for a single timestep.
    """

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self.trainable_weights = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [0]:
def model_gru_attn():
  inp = Input(shape=(max_len,))
  x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  x = SpatialDropout1D(0.1)(x)
  x = Bidirectional(CuDNNLSTM(50, return_sequences=True))(x)
  x = Bidirectional(CuDNNGRU(50, return_sequences=True))(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  last = Lambda(lambda t: t[:, -1])(x)
  attn = AttentionWeightedAverage()(x)
  conc = concatenate([avg_pool,  max_pool,last,attn])
  outp = Dense(1, activation="sigmoid")(conc)    
  model = Model(inputs=inp, outputs=outp)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer=opt, metrics=['accuracy'])
  return model

##TCN

In [0]:
from tcn import TCN

def wave_net_activation(x):
    # type: (Layer) -> Layer
    """This method defines the activation used for WaveNet
    described in https://deepmind.com/blog/wavenet-generative-model-raw-audio/
    Args:
        x: The layer we want to apply the activation to
    Returns:
        A new layer with the wavenet activation applied
    """
    tanh_out = Activation('tanh')(x)
    sigm_out = Activation('sigmoid')(x)
    return keras.layers.multiply([tanh_out, sigm_out])

def model_tcn(embedding_matrix):
    
    inp = Input(shape=(max_len,))
    x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
    x = SpatialDropout1D(0.1)(x)
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True ,name = 'tnc1')(x)
    x = wave_net_activation(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, name = 'tnc2')(x)
    x = wave_net_activation(x)
    #x = TCN(32,dilations = [1, 2, 4], return_sequences=True, activation = 'wavenet',name = 'tnc3')(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dense(32, activation="relu")(conc)

    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss=BinaryFocalLoss(gamma=2), optimizer=opt, metrics=['accuracy'])
    
    return model

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights=dict(enumerate(class_weights))

## VDCNN

In [0]:
#https://github.com/mukesh-mehta/VDCNN/blob/master/toxic.ipynb
def ConvolutionalBlock(input_shape, num_filters):
    model=Sequential()

    #1st conv layer
    model.add(Conv1D(filters = num_filters, kernel_size = 3, strides = 1, padding = "same", input_shape = input_shape))
    model.add(BatchNormalization())
    model.add(Activation("relu"))

    #2nd conv layer
    model.add(Conv1D(filters = num_filters, kernel_size = 3, strides = 1, padding = "same"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))

    return model

#https://www.tensorflow.org/api_docs/python/tf/nn/top_k
def top_kmax(x):
    x=tf.transpose(x, [0, 2, 1])
    k_max = tf.nn.top_k(x, k=top_k)
    return tf.reshape(k_max[0], (-1, num_filters[-1]*top_k))

def conv_shape(conv):
    return conv.get_shape().as_list()[1:]


In [0]:
def model_vdcnn(num_filters, num_classes, sequence_max_length, num_chars, embedding_size, top_k, learning_rate=0.001):
    
    inputs=Input(shape=(sequence_max_length, ), dtype='int32', name='input')
    
    embedded_seq = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inputs)
    embedded_seq = BatchNormalization()(embedded_seq)
    #1st Layer
    conv = Conv1D(filters=64, kernel_size=3, strides=2, padding="same")(embedded_seq)
    
    #ConvBlocks
    for i in range(len(num_filters)):
        conv = ConvolutionalBlock(conv_shape(conv), num_filters[i])(conv)
        conv = MaxPooling1D(pool_size=3, strides=2, padding="same")(conv)
        
    def _top_k(x):
        x = tf.transpose(x, [0, 2, 1])
        k_max = tf.nn.top_k(x, k=top_k)
        return tf.reshape(k_max[0], (-1, num_filters[-1] * top_k))
    
    k_max = Lambda(_top_k, output_shape=(num_filters[-1] * top_k,))(conv)
    
    #fully connected layers
    # in original paper they didn't used dropouts
    fc1=Dense(100, activation='relu', kernel_initializer='he_normal')(k_max)
    fc1=Dropout(0.3)(fc1)
    fc2=Dense(20, activation='relu', kernel_initializer='he_normal')(fc1)
    fc2=Dropout(0.3)(fc2)
    out=Dense(1, activation='sigmoid')(fc2)
    
    
    model = Model(inputs=inputs, outputs=out)
    model.compile(optimizer=opt, loss=BinaryFocalLoss(gamma=2), metrics=['accuracy'])
    
    return model

In [0]:
num_filters = [64, 128, 256, 512]
vdcnn_model=model_vdcnn(num_filters=num_filters, num_classes=1,num_chars=69, sequence_max_length=max_len,embedding_size=16,top_k=3)

##KIM

In [0]:
conv_filters=128
def model_kim():
  inp = Input(shape=(max_len,))
  emb = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  # Specify each convolution layer and their kernel siz i.e. n-grams 
  conv1_1 = Conv1D(filters=conv_filters, kernel_size=3)(emb)
  btch1_1 = BatchNormalization()(conv1_1)
  drp1_1  = Dropout(0.2)(btch1_1)
  actv1_1 = Activation('relu')(drp1_1)
  glmp1_1 = GlobalMaxPooling1D()(actv1_1)

  conv1_2 = Conv1D(filters=conv_filters, kernel_size=4)(emb)
  btch1_2 = BatchNormalization()(conv1_2)
  drp1_2  = Dropout(0.2)(btch1_2)
  actv1_2 = Activation('relu')(drp1_2)
  glmp1_2 = GlobalMaxPooling1D()(actv1_2)

  conv1_3 = Conv1D(filters=conv_filters, kernel_size=5)(emb)
  btch1_3 = BatchNormalization()(conv1_3)
  drp1_3  = Dropout(0.2)(btch1_3)
  actv1_3 = Activation('relu')(drp1_3)
  glmp1_3 = GlobalMaxPooling1D()(actv1_3)

  conv1_4 = Conv1D(filters=conv_filters, kernel_size=6)(emb)
  btch1_4 = BatchNormalization()(conv1_4)
  drp1_4  = Dropout(0.2)(btch1_4)
  actv1_4 = Activation('relu')(drp1_4)
  glmp1_4 = GlobalMaxPooling1D()(actv1_4)

  # Gather all convolution layers
  cnct = concatenate([glmp1_1, glmp1_2, glmp1_3, glmp1_4], axis=1)
  drp1 = Dropout(0.2)(cnct)

  dns1  = Dense(32, activation='relu')(drp1)
  btch1 = BatchNormalization()(dns1)
  drp2  = Dropout(0.2)(btch1)

  out = Dense(1, activation='sigmoid')(drp2)   
  model = Model(inputs=inp, outputs=out)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer=opt, metrics=['accuracy'])
  return model


## CharCNN

In [53]:

# parameter
input_size = 200
vocab_size = len(ch_tokenize.word_index)
embedding_size = len(alphabet)+1
conv_layers = [[256, 7, 3],
               [256, 7, 3],
               [256, 3, -1],
               [256, 3, -1],
               [256, 3, -1],
               [256, 3, 3]]

fully_connected_layers = [1024, 1024]
dropout_p = 0.5


# Embedding weights
embedding_weights = []  
embedding_weights.append(np.zeros(vocab_size))  

for char, i in ch_tokenize.word_index.items():  
    onehot = np.zeros(vocab_size)
    onehot[i - 1] = 1
    embedding_weights.append(onehot)

embedding_weights = np.array(embedding_weights)
print('Load')

# Embedding layer Initialization
embedding_layer = Embedding(vocab_size + 1,
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights])

# Model Construction
# Input
inputs = Input(shape=(input_size,), name='input', dtype='int64')  # shape=(?, 1014)
# Embedding
x = embedding_layer(inputs)
# Conv
for filter_num, filter_size, pooling_size in conv_layers:
    x = Conv1D(filter_num, filter_size)(x)
    x = Activation('relu')(x)
    if pooling_size != -1:
        x = MaxPooling1D(pool_size=pooling_size)(x)  # Final shape=(None, 34, 256)
x = Flatten()(x)  # (None, 8704)
# Fully connected layers
for dense_size in fully_connected_layers:
    x = Dense(dense_size, activation='relu')(x)  # dense_size == 1024
    x = Dropout(dropout_p)(x)
# Output Layer
predictions = Dense(1, activation='sigmoid')(x)
# Build model
char_model = Model(inputs=inputs, outputs=predictions)
char_model.compile(loss=BinaryFocalLoss(gamma=2), optimizer=opt, metrics=['accuracy'])  # Adam, categorical_crossentropy

Load


# Train model

##GRU

In [66]:
gru_model=model_gru()
gru_model.fit(train_sequences,y_train,batch_size=batch_size,epochs=10,verbose=2,class_weight=class_weights,validation_data=(val_sequences,y_val),callbacks= [clr,])

Train on 6999 samples, validate on 999 samples
Epoch 1/10
 - 8s - loss: 0.0877 - acc: 0.8647 - val_loss: 0.0672 - val_acc: 0.9019
Epoch 2/10
 - 7s - loss: 0.0591 - acc: 0.9164 - val_loss: 0.0639 - val_acc: 0.9049
Epoch 3/10
 - 7s - loss: 0.0504 - acc: 0.9264 - val_loss: 0.0691 - val_acc: 0.9099
Epoch 4/10
 - 7s - loss: 0.0436 - acc: 0.9380 - val_loss: 0.0736 - val_acc: 0.9019
Epoch 5/10
 - 7s - loss: 0.0358 - acc: 0.9494 - val_loss: 0.0745 - val_acc: 0.9139
Epoch 6/10
 - 7s - loss: 0.0302 - acc: 0.9560 - val_loss: 0.0846 - val_acc: 0.9129
Epoch 7/10
 - 7s - loss: 0.0217 - acc: 0.9707 - val_loss: 0.0962 - val_acc: 0.8909
Epoch 8/10
 - 7s - loss: 0.0154 - acc: 0.9807 - val_loss: 0.1286 - val_acc: 0.9169
Epoch 9/10
 - 7s - loss: 0.0117 - acc: 0.9846 - val_loss: 0.1427 - val_acc: 0.9099
Epoch 10/10
 - 7s - loss: 0.0075 - acc: 0.9903 - val_loss: 0.1834 - val_acc: 0.9129


## GRU attention

In [67]:
attn_gru_model=model_gru_attn()
attn_gru_model.fit(train_sequences,y_train,batch_size=batch_size,epochs=10,verbose=2,class_weight=class_weights,validation_data=(val_sequences,y_val),callbacks= [clr,])

Train on 6999 samples, validate on 999 samples
Epoch 1/10
 - 8s - loss: 0.1113 - acc: 0.8374 - val_loss: 0.0681 - val_acc: 0.9069
Epoch 2/10
 - 7s - loss: 0.0644 - acc: 0.9061 - val_loss: 0.0696 - val_acc: 0.9009
Epoch 3/10
 - 7s - loss: 0.0528 - acc: 0.9253 - val_loss: 0.0644 - val_acc: 0.9079
Epoch 4/10
 - 7s - loss: 0.0461 - acc: 0.9340 - val_loss: 0.0627 - val_acc: 0.9169
Epoch 5/10
 - 7s - loss: 0.0405 - acc: 0.9417 - val_loss: 0.0660 - val_acc: 0.9179
Epoch 6/10
 - 7s - loss: 0.0336 - acc: 0.9527 - val_loss: 0.0737 - val_acc: 0.9119
Epoch 7/10
 - 7s - loss: 0.0269 - acc: 0.9610 - val_loss: 0.0896 - val_acc: 0.9129
Epoch 8/10
 - 7s - loss: 0.0248 - acc: 0.9639 - val_loss: 0.0904 - val_acc: 0.9109
Epoch 9/10
 - 7s - loss: 0.0172 - acc: 0.9774 - val_loss: 0.1167 - val_acc: 0.9029
Epoch 10/10
 - 7s - loss: 0.0137 - acc: 0.9803 - val_loss: 0.1205 - val_acc: 0.8939


##TCN

In [68]:
tcn_model=model_tcn(embeddings)
tcn_model.fit(train_sequences,y_train,batch_size=batch_size,epochs=10,verbose=2,class_weight=class_weights,validation_data=(val_sequences,y_val),callbacks= [clr,])

Train on 6999 samples, validate on 999 samples
Epoch 1/10
 - 9s - loss: 0.1397 - acc: 0.7825 - val_loss: 0.1225 - val_acc: 0.8208
Epoch 2/10
 - 4s - loss: 0.1334 - acc: 0.7985 - val_loss: 0.1240 - val_acc: 0.8208
Epoch 3/10
 - 4s - loss: 0.1318 - acc: 0.7985 - val_loss: 0.1228 - val_acc: 0.8208
Epoch 4/10
 - 4s - loss: 0.1320 - acc: 0.7985 - val_loss: 0.1234 - val_acc: 0.8208
Epoch 5/10
 - 4s - loss: 0.1321 - acc: 0.7985 - val_loss: 0.1232 - val_acc: 0.8208
Epoch 6/10
 - 4s - loss: 0.1316 - acc: 0.7985 - val_loss: 0.1234 - val_acc: 0.8208
Epoch 7/10
 - 4s - loss: 0.1320 - acc: 0.7985 - val_loss: 0.1238 - val_acc: 0.8208
Epoch 8/10
 - 4s - loss: 0.1314 - acc: 0.7985 - val_loss: 0.1229 - val_acc: 0.8208
Epoch 9/10
 - 4s - loss: 0.1314 - acc: 0.7985 - val_loss: 0.1226 - val_acc: 0.8208
Epoch 10/10
 - 4s - loss: 0.1314 - acc: 0.7985 - val_loss: 0.1228 - val_acc: 0.8208


##VDCNN

In [77]:
vdcnn_model.fit(train_sequences,y_train,batch_size=batch_size,epochs=10,verbose=2,class_weight=class_weights,validation_data=(val_sequences,y_val),callbacks= [clr,])

Train on 6999 samples, validate on 999 samples
Epoch 1/10
 - 14s - loss: 0.1313 - acc: 0.7985 - val_loss: 0.1228 - val_acc: 0.8208
Epoch 2/10
 - 14s - loss: 0.1315 - acc: 0.7985 - val_loss: 0.1226 - val_acc: 0.8208
Epoch 3/10
 - 14s - loss: 0.1310 - acc: 0.7985 - val_loss: 0.1230 - val_acc: 0.8208
Epoch 4/10
 - 14s - loss: 0.1312 - acc: 0.7985 - val_loss: 0.1234 - val_acc: 0.8208
Epoch 5/10
 - 14s - loss: 0.1313 - acc: 0.7985 - val_loss: 0.1230 - val_acc: 0.8208
Epoch 6/10
 - 14s - loss: 0.1312 - acc: 0.7985 - val_loss: 0.1223 - val_acc: 0.8208
Epoch 7/10
 - 14s - loss: 0.1315 - acc: 0.7985 - val_loss: 0.1229 - val_acc: 0.8208
Epoch 8/10
 - 14s - loss: 0.1305 - acc: 0.7985 - val_loss: 0.1218 - val_acc: 0.8208
Epoch 9/10
 - 14s - loss: 0.1297 - acc: 0.7985 - val_loss: 0.1207 - val_acc: 0.8208
Epoch 10/10
 - 14s - loss: 0.1268 - acc: 0.7985 - val_loss: 0.1164 - val_acc: 0.8208


##KIM

In [80]:
kim_model=model_kim()
kim_model.fit(train_sequences,y_train,batch_size=batch_size,epochs=10,verbose=2,class_weight=class_weights,validation_data=(val_sequences,y_val),callbacks= [clr,])

Train on 6999 samples, validate on 999 samples
Epoch 1/10
 - 6s - loss: 0.1326 - acc: 0.7925 - val_loss: 0.0825 - val_acc: 0.8699
Epoch 2/10
 - 4s - loss: 0.0825 - acc: 0.8748 - val_loss: 0.0761 - val_acc: 0.8829
Epoch 3/10
 - 4s - loss: 0.0700 - acc: 0.8976 - val_loss: 0.0664 - val_acc: 0.9019
Epoch 4/10
 - 4s - loss: 0.0631 - acc: 0.9063 - val_loss: 0.0684 - val_acc: 0.8969
Epoch 5/10
 - 4s - loss: 0.0573 - acc: 0.9154 - val_loss: 0.0640 - val_acc: 0.8989
Epoch 6/10
 - 4s - loss: 0.0523 - acc: 0.9248 - val_loss: 0.0748 - val_acc: 0.8969
Epoch 7/10
 - 4s - loss: 0.0473 - acc: 0.9308 - val_loss: 0.0687 - val_acc: 0.9079
Epoch 8/10
 - 4s - loss: 0.0425 - acc: 0.9410 - val_loss: 0.0664 - val_acc: 0.9119
Epoch 9/10
 - 4s - loss: 0.0381 - acc: 0.9458 - val_loss: 0.0792 - val_acc: 0.9049
Epoch 10/10
 - 4s - loss: 0.0338 - acc: 0.9521 - val_loss: 0.0671 - val_acc: 0.9139


## CharCNN

In [81]:
char_model.fit(train_data,y_train,batch_size=batch_size,epochs=10,verbose=2,class_weight=class_weights,validation_data=(val_data,y_val),callbacks= [clr,])

Train on 6999 samples, validate on 999 samples
Epoch 1/10
 - 5s - loss: 3.3051 - acc: 0.7975 - val_loss: 2.8880 - val_acc: 0.8208
Epoch 2/10
 - 3s - loss: 3.3254 - acc: 0.7985 - val_loss: 2.8880 - val_acc: 0.8208
Epoch 3/10
 - 3s - loss: 3.3342 - acc: 0.7985 - val_loss: 2.8880 - val_acc: 0.8208
Epoch 4/10
 - 3s - loss: 3.3148 - acc: 0.7985 - val_loss: 2.8880 - val_acc: 0.8208
Epoch 5/10
 - 3s - loss: 3.3265 - acc: 0.7985 - val_loss: 2.8880 - val_acc: 0.8208
Epoch 6/10
 - 3s - loss: 3.3236 - acc: 0.7985 - val_loss: 2.8880 - val_acc: 0.8208
Epoch 7/10
 - 2s - loss: 3.3100 - acc: 0.7985 - val_loss: 2.8880 - val_acc: 0.8208
Epoch 8/10
 - 3s - loss: 3.3298 - acc: 0.7985 - val_loss: 2.8880 - val_acc: 0.8208
Epoch 9/10
 - 2s - loss: 3.3428 - acc: 0.7985 - val_loss: 2.8880 - val_acc: 0.8208
Epoch 10/10
 - 2s - loss: 3.3106 - acc: 0.7985 - val_loss: 2.8880 - val_acc: 0.8208


# Validate Models

##GRU

In [83]:
y_pred = gru_model.predict(val_sequences, batch_size=batch_size, verbose=1)
y_pred = (y_pred > 0.5)
print(classification_report(y_val, y_pred))
gru_f1=f1_score(y_val, y_pred,average='macro')

999/999 [==============================] - 0s 368us/step
              precision    recall  f1-score   support

           0       0.93      0.97      0.95       820
           1       0.83      0.64      0.73       179

    accuracy                           0.91       999
   macro avg       0.88      0.81      0.84       999
weighted avg       0.91      0.91      0.91       999



## GRU attention

In [84]:
y_pred = attn_gru_model.predict(val_sequences, batch_size=batch_size, verbose=1)
y_pred = (y_pred > 0.5)
print(classification_report(y_val, y_pred))
gru_atten_f1=f1_score(y_val, y_pred,average='macro')

999/999 [==============================] - 2s 2ms/step
              precision    recall  f1-score   support

           0       0.94      0.93      0.94       820
           1       0.70      0.72      0.71       179

    accuracy                           0.89       999
   macro avg       0.82      0.83      0.82       999
weighted avg       0.90      0.89      0.89       999



##TCN

In [85]:
y_pred = tcn_model.predict(val_sequences, batch_size=batch_size, verbose=1)
y_pred = (y_pred > 0.5)
print(classification_report(y_val, y_pred))
tcn_f1=f1_score(y_val, y_pred,average='macro')

999/999 [==============================] - 1s 1ms/step
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       820
           1       0.00      0.00      0.00       179

    accuracy                           0.82       999
   macro avg       0.41      0.50      0.45       999
weighted avg       0.67      0.82      0.74       999



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## VDCNN

In [86]:
y_pred = vdcnn_model.predict(val_sequences, batch_size=batch_size, verbose=1)
y_pred = (y_pred > 0.5)
print(classification_report(y_val, y_pred))
vdcnn_f1=f1_score(y_val, y_pred,average='macro')

999/999 [==============================] - 3s 3ms/step
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       820
           1       0.00      0.00      0.00       179

    accuracy                           0.82       999
   macro avg       0.41      0.50      0.45       999
weighted avg       0.67      0.82      0.74       999



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##KIM

In [87]:
y_pred = kim_model.predict(val_sequences, batch_size=batch_size, verbose=1)
y_pred = (y_pred > 0.5)
print(classification_report(y_val, y_pred))
kim_f1=f1_score(y_val, y_pred,average='macro')

999/999 [==============================] - 1s 1ms/step
              precision    recall  f1-score   support

           0       0.93      0.97      0.95       820
           1       0.83      0.65      0.73       179

    accuracy                           0.91       999
   macro avg       0.88      0.81      0.84       999
weighted avg       0.91      0.91      0.91       999



## CharCNN

In [89]:
y_pred = char_model.predict(val_data, batch_size=batch_size, verbose=1)
y_pred = (y_pred > 0.5)
print(classification_report(y_val, y_pred))
char_f1=f1_score(y_val, y_pred,average='macro')

999/999 [==============================] - 1s 1ms/step
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       820
           1       0.00      0.00      0.00       179

    accuracy                           0.82       999
   macro avg       0.41      0.50      0.45       999
weighted avg       0.67      0.82      0.74       999



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Result

In [90]:
print('f1 score of count vec' ,cv_f1)
print('f1 score of word tfidf' ,tfw_f1)
print('f1 score of char tfidf' ,tfc_f1)

print('f1 score of RNN' ,gru_f1)
print('f1 score of gated attention', gru_atten_f1)
print('f1 score of tcn',tcn_f1)
print('f1 score of vdcnn', vdcnn_f1)
print('f1 score of kim',kim_f1)
print('f1 score of char cnn',char_f1)

f1 score of count vec 0.7795286909724823
f1 score of word tfidf 0.7994039900389771
f1 score of char tfidf 0.8274204946996466
f1 score of RNN 0.8368985713401028
f1 score of gated attention 0.8219598638815285
f1 score of tcn 0.4507971412864211
f1 score of vdcnn 0.4507971412864211
f1 score of kim 0.8399992550655543
f1 score of char cnn 0.4507971412864211


# Ensemble

we will choose only those, having f1 greater than 0.7. Once they are selected, we will decode labels, and keep label according to mod

In [94]:
y_test_cv = cv_classifier.predict_proba(X_test_cv)[:,1]
y_test_tfw = tfw_classifier.predict_proba(test_word_features)[:,1]
y_test_tfc = tfc_classifier.predict_proba(test_char_features)[:,1]

y_test_gru = gru_model.predict(test_sequences, batch_size=batch_size, verbose=1).ravel()
y_test_gru_atten = attn_gru_model.predict(test_sequences, batch_size=batch_size, verbose=1).ravel()
# y_test_tcn = tcn_model.predict(X_test_sequences, batch_size=batch_size, verbose=1).ravel()
# y_test_vdcnn = vdcnn_model.predict(X_test_sequences, batch_size=batch_size, verbose=1).ravel()
y_test_kim = kim_model.predict(test_sequences, batch_size=batch_size, verbose=1).ravel()
# y_test_char = char_model.predict(X_test_sequences, batch_size=batch_size, verbose=1).ravel()
# # # 

2000/2000 [==============================] - 0s 168us/step


In [96]:
f=np.array([0.78,0.80,0.83,0.84,0.83,0.84])
f/np.sum(f)

array([0.15853659, 0.16260163, 0.16869919, 0.17073171, 0.16869919,
       0.17073171])

In [0]:
#i am bit changing the distribution in order to give more weightage to ml than dl
y_test=0.15*y_test_cv  + 0.18*y_test_tfw + 0.18*y_test_tfc + 0.17*y_test_gru + 0.16*y_test_gru_atten + 0.17*y_test_kim

In [0]:
def decode(y_test):
  y_test[y_test>0.5]=1
  y_test[y_test<0.5]=0
  y_test=y_test.astype('int16').ravel()

  y_test=le.inverse_transform(y_test)
  y_test=pd.DataFrame(y_test,columns=['label'])
  y_test=pd.concat([ids, y_test['label']], axis=1)
  return y_test

In [100]:
y_test=decode(y_test)
y_test.head()



,id,label
0,8001,OFF
1,8002,NOT
2,8003,NOT
3,8004,OFF
4,8005,NOT


# Submit file

In [0]:
y_test.to_csv('/content/weighted_clr_focalloss.csv',index=False,header=None)


# Reset

In [0]:
# import keras
# keras.backend.clear_session()